# Pattern Recognition - Project #2

*   **Course:** Pattern Recognition - Fall 2020
*   **Instructor:** [Dr. Morteza Analoui](https://www.scopus.com/authid/detail.uri?authorId=16835800400)
*   **Teaching Assistants:** Seyed Hassan Tabatabaei, Pedram Dadkhah
*   **Student:** [Parsa Abbasi](https://parsa-abbasi.ir/)
*   ***Iran University of Science and Technology (IUST)***




## Libraries

In [1]:
# General
from scipy.io import arff
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tqdm.notebook import trange, tqdm

# Visualization
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.neighbors import NearestNeighbors

# Metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

## 1) Dataset

The dataset we will use in this project is called ***habermanImb*** and the name refers to the *S.J. Haberman* research in 1976.   
The dataset was obtained from a study conducted between 1958 and 1970 at the University of *Chicago's Billings Hospital* on the survival of patients after breast cancer surgery [[1]](https://lib.ugent.be/fulltxt/RUG01/002/163/676/RUG01-002163676_2014_0001_AC.pdf).   
It contains two classes that are already imbalanced. The <font color='green'>positive</font> class means that the patient survived 5 years or longer, while the <font color='red'>negative</font> class means that the patient passed away within 5 years.

**Attributes (Columns) :**
*   *Age:* Age of patient at the time of operation
*   *Year:* Patient's year of operation (year - 1900)
*   *Positive:* Number of positive axillary nodes detected
    *    If lymph nodes have some cancer cells in them, they are called positive [[2]](https://www.breastcancer.org/symptoms/diagnosis/lymph_nodes).
    *   A positive axillary lymph node is a lymph node in the area of the armpit (axilla) to which cancer has spread [[3]](https://en.wikipedia.org/wiki/Positive_axillary_lymph_node#:~:text=A%20positive%20axillary%20lymph%20node,whether%20cancer%20cells%20are%20present). 
*   *Class:* Survival status
    *   <font color='blue'>Positive = +1:</font> The patient survived 5 years or longer
    *   <font color='red'>Negative = -1:</font> The patient passed away within 5 years





### 1.1) Loading

In [2]:
# The paths to dataset files from my google drive
files_addresses = [
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-1tra.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-1tst.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-2tra.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-2tst.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-3tra.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-3tst.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-4tra.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-4tst.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-5tra.arff',
                   '/content/drive/MyDrive/Dataset/HabermanImbalanced/habermanImb-5-5tst.arff'
]

In [3]:
# Creating a dataframe that contains all the extracted data
data = pd.DataFrame(columns=['Age', 'Year', 'Positive', 'Class'])

In [4]:
# Read each of the dataset files
for address in files_addresses:
  file_arff = arff.loadarff(address)
  file_df = pd.DataFrame(file_arff[0])
  data = data.append(file_df,ignore_index=True)

In [5]:
# Take a look at the data
data[:10]

,Age,Year,Positive,Class
0,38.0,59.0,2.0,b'negative'
1,39.0,63.0,4.0,b'negative'
2,49.0,62.0,1.0,b'negative'
3,56.0,67.0,0.0,b'negative'
4,64.0,58.0,0.0,b'negative'
5,55.0,69.0,22.0,b'negative'
6,45.0,66.0,0.0,b'positive'
7,52.0,61.0,0.0,b'negative'
8,61.0,65.0,8.0,b'negative'
9,54.0,62.0,0.0,b'negative'


### 1.2) Encode classes

In [6]:
# Change the class labels to a real value
data.loc[data['Class'] == b'negative', 'Class'] = -1
data.loc[data['Class'] == b'positive', 'Class'] = 1

In [7]:
data[:10]

,Age,Year,Positive,Class
0,38.0,59.0,2.0,-1
1,39.0,63.0,4.0,-1
2,49.0,62.0,1.0,-1
3,56.0,67.0,0.0,-1
4,64.0,58.0,0.0,-1
5,55.0,69.0,22.0,-1
6,45.0,66.0,0.0,1
7,52.0,61.0,0.0,-1
8,61.0,65.0,8.0,-1
9,54.0,62.0,0.0,-1


In [8]:
# Extract positive and negative samples separately

positive_samples = data.loc[data['Class'] == 1]
negative_samples = data.loc[data['Class'] == -1]

positive_counts = len(positive_samples)
negative_counts = len(negative_samples)

dif_counts = negative_counts - positive_counts
ratio = negative_counts/positive_counts
num_data = len(data)

print("The number of data:", num_data)
print("The number of positive samples:", positive_counts)
print("The number of negative samples:", negative_counts)
print("The difference between the majority class to the minority class:", dif_counts)
print("The ratio of majority class to the minority class:", ratio)

The number of data: 1530
The number of positive samples: 405
The number of negative samples: 1125
The difference between the majority class to the minority class: 720
The ratio of majority class to the minority class: 2.7777777777777777


In [9]:
duplicates_num = len(data)-len(data.drop_duplicates())
print('Number of duplicate data:', duplicates_num)
print('Number of unique data:', len(data.drop_duplicates()))

Number of duplicate data: 1241
Number of unique data: 289


Most of the examples in the dataset are repetitive. The question is should we remove the duplicate samples? In most of the situation, we should do that, but here as the data related to diseases, it's more cautious to suppose each of them is identical.

In [10]:
# 3D visualization
px.scatter_3d(data, x='Positive', y='Age', z='Year',
              title='Original Data', color='Class',
              color_discrete_sequence=px.colors.qualitative.Set1)

### 1.3) Normalization

In [11]:
def min_max_normalization(dataframe, column):
  # Finding the minimum and maximum values of the feature
  minimum_value = dataframe[column].min()
  maximum_value = dataframe[column].max()
  # Subtracting the minimum value of the feature
  numerator = dataframe[column] - minimum_value
  # Dividing by the range
  denominator = maximum_value - minimum_value
  # Putting all together
  dataframe[column] = numerator/denominator

In [12]:
# Apply min-max normalization on the Age, Year, and Positive columns
min_max_normalization(data, 'Age')
min_max_normalization(data, 'Year')
min_max_normalization(data, 'Positive')

In [13]:
data.head()

,Age,Year,Positive,Class
0,0.150943,0.090909,0.038462,-1
1,0.169811,0.454545,0.076923,-1
2,0.358491,0.363636,0.019231,-1
3,0.490566,0.818182,0.000000,-1
4,0.641509,0.000000,0.000000,-1


In [14]:
# 3D visualization
px.scatter_3d(data, x='Positive', y='Age', z='Year',
              title='Normalized Data', color='Class',
              color_discrete_sequence=px.colors.qualitative.Set1)

### 1.4) Prepare data

Splitting features from the label.

In [15]:
# Splitting feature vectors and class labels
X = data[['Age','Year','Positive']].values
y = data['Class'].values
y = y.astype('int')

### 1.5) K-fold

In [16]:
# Setting 5-fold
five_fold = KFold(n_splits=5, random_state=42, shuffle=True)

### 1.6) Visualization and Metric functions

In [17]:
def get_metrics(true_labels, predicted_labels):
  accuracy = accuracy_score(true_labels, predicted_labels)
  precision = precision_score(true_labels, predicted_labels)
  recall = recall_score(true_labels, predicted_labels)
  fscore = f1_score(true_labels, predicted_labels, labels=np.unique(predicted_labels))
  result = {'Precision': precision, 'Recall': recall, 'F1score':fscore, 'Accuracy':accuracy}
  cm = confusion_matrix(true_labels, predicted_labels).ravel()
  # Output = (tn, fp, fn, tp)
  result['TN'], result['FP'], result['FN'], result['TP'] = cm
  return result

In [18]:
class LossPlotter:
  def __init__(self, title, xaxis, yaxis):
    self.fig = go.Figure()
    self.fig.update_layout(title=title,
                    xaxis_title=xaxis,
                    yaxis_title=yaxis
                    )

  def add_scatter(self, losses, name):
    fig_x = list(range(len(losses)))
    fig_x = [i+1 for i in fig_x]
    self.fig = self.fig.add_trace(go.Scatter(x=fig_x, 
                                        y=losses,
                                        name=name))

  def show(self):
    self.fig.show()

In [19]:
class RocPlotter:
  def __init__(self, title):
    self.fig = go.Figure()
    self.fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )
    self.fig.update_layout(title=title)

  def add_scatter(self, true_label, pred_scores, name):
    fpr, tpr, thresholds = roc_curve(true_label, pred_scores)
    auc_score = roc_auc_score(true_label, pred_scores)
    scatter_name = f"{name} (AUC={auc_score:.4f})"
    self.fig.add_trace(go.Scatter(x=fpr, y=tpr, name=scatter_name))
    return auc_score

  def show(self):
    self.fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    self.fig.show()

In [20]:
def plot_histogram(true_label, pred_scores, title):
  fpr, tpr, thresholds = roc_curve(true_label, pred_scores)

  # The histogram of scores compared to true labels
  fig_hist = px.histogram(
      x=pred_scores, color=true_label, nbins=50,
      labels=dict(color='True Labels', x='Score')
  )
  fig_hist.update_layout(title=title)
  fig_hist.show()

The following function takes the labels and ensemble values then compute exponential loss.

In [21]:
def ensemble_loss_binary(true_labels, pred_labels, values):

  # The number of samples
  m = len(true_labels)

  # The exponential loss value
  loss = 0

  # For each sample
  for index, hfin in enumerate(values):
    current_loss = np.exp(-true_labels[index]*hfin)
    loss += current_loss

  loss = (1/m)*loss

  return loss

### 1.7) Hyperparameter and Variables

The hyperparameters we should set are following :

*   **max_depth:** It defines the maximum depth for the decision tree. If it will set to 1, then the decision tree act like a stump.
*   **max_features:** It defines the maximum features that the decision tree should be considered. We usually set it equal to 1 in Boosting algorithms.
*   **k_in_knn:** The number of nearest neighbors. As in this particular dataset, there are many duplicate examples, and also we need a good diversity in SMOTE algorithm this hyperparameter was set to 20.
*   **smote_ratio:** The SMOTE ratio when creating synthetic examples. For example, if it was set to 200, then the number of synthetic examples will be equal to double the minority class size.
*   **T:** A list of the number of hypothesis/classifier we should make in each boosting algorithm.



In [22]:
max_depth = 1
max_features = 1
k_in_knn = 20
smote_ratio = 150

# Each value of T is the number of classifiers
T = [10, 50, 100]

# A dataframe containing the results for 5-fold
five_fold_results = pd.DataFrame(
    columns=['Algorithm', 'T', 'Loss', 'Precision', 'Recall',
             'F1score', 'AUC', 'Accuracy', 'TN', 'FP', 'FN', 'TP']
    )

# A dataframe containing the results for whole training data
train_results = pd.DataFrame(
    columns=['Algorithm', 'T', 'Loss', 'Precision', 'Recall',
             'F1score', 'AUC', 'Accuracy', 'TN', 'FP', 'FN', 'TP']
    )

## 2) AdaBoost.M2

**AdaBoost** is a practical and easy to implement boosting algorithm. It can be used to significantly reduce the error of any learning algorithm that consistently generates classifiers whose performance is a little better than random guessing [[4]](https://cseweb.ucsd.edu/~yfreund/papers/boostingexperiments.pdf).

In multi-label problems with $k>2$ the requirement that the error be less than $1/2$ is may often be hard to meet in **AdaBoost.M1**, but in the second version of the algorithm, **AdaBoost.M2**, this difficulty solved by using a more sophisticated error measure called *pseudo-loss*.   
Pseudo-loss is a method for forcing a learning algorithm of multi-label concepts to concentrate on the labels that are hardest to discriminate.

The pseudo-code shown in the bellow is from a [paper](https://arxiv.org/pdf/1708.03704.pdf) by *Alan Mosca* and *George D. Magoulas*.

![picture](https://drive.google.com/uc?id=1smHu06WuANhTiJ4EqO0XIrn7ud5PeuAV)

**Inputs:**
*   $X$: sequence of $m$ samples feature vectors $[x_1, x_2, ..., x_m]$
*   $y$: sequence of $m$ samples labels $[y_1, y_2, ..., y_m]$
*   $T$: an integer specifying the number of iterations


### 2.1) Algorithm implementation

In [23]:
class AdaBoostM2:
  """
    Functions:
      run(self) -> Run the algorithm of AdaBoost.M2
      predict(self, input) -> Predict the labels for given samples using created ensemble
      pseudo_losses(self) -> The pseudo-loss for each hypothesis
  """


  # Initilizition
  def __init__(self, X, y, T):
    """
      X: Feature vectors
      y: Labels
      m: Number of samples
      T: Number of iterations/classifiers/hypothesis
      H: Ensemble of classifiers
      beta_list: A list containing the beta value for each classifier
      loss_list: A list containing the pseudo-loss of each classifier
    """
    self.X = X
    self.y = y
    self.m = len(X)
    self.T = T
    self.H = []
    self.beta_list = []
    self.loss_list = []
    self.index_to_label = {}
    self.label_to_index = {}
  

  # Running the AdaBoost.M2 algorithm
  def run(self):

    # Initilize distribution
    D_t = (1./self.m) * np.ones(self.m)

    for t in trange(self.T):

      # Train a new classifier on training data with distibution D_t
      weak_learn = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
      weak_learn.fit(self.X, self.y, sample_weight=D_t)
      
      # Get predictions and probabilities for the training data
      predictions = weak_learn.predict(self.X)
      probabilities = weak_learn.predict_proba(self.X)

      if t == 0:
        for index, value in enumerate(weak_learn.classes_):
          self.index_to_label[index] = value
          self.label_to_index[value] = index
      
      # Compute pseudo-loss
      pseudo_loss = 0

      # For each sample
      for index, true_label in enumerate(self.y):
        
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]

        # For each wrong label
        current_loss = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            current_loss += D_t[index] * (1 - true_prob + wrong_prob)
        # Add it to pseudo_loss
        pseudo_loss += current_loss

      # Final step to compute pseudo-loss
      pseudo_loss = (1/2)*pseudo_loss

      # Compute beta
      beta = pseudo_loss / (1 - pseudo_loss)
      
      # Update distribution
      for index, value in enumerate(D_t):
        predicted_label = predictions[index]
        true_label = self.y[index]
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]
        # Probability of assigning the wrong label
        power = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            power += (1 + true_prob - wrong_prob)
        power = (1/2) * power
        D_t[index] = (value) * pow(beta,power)
      
      # Normalization factor
      Z_t = np.sum(D_t)
      D_t = D_t / Z_t

      # Append weak learner to the ensemble
      self.H.append(weak_learn)
      # Append computed loss of the weak learner
      self.loss_list.append(pseudo_loss)
      # Append computed beta of the weak learner
      self.beta_list.append(beta)
  
      # print('t =', t, '--> pseudo-loss =', pseudo_loss)
    
    # Convert beta and loss lists to numpy array
    self.beta_list = np.array(self.beta_list)
    self.loss_list = np.array(self.loss_list)


  def predict(self, input):
    # Sum of the probabilities made by each hypothesis for a given sample and label
    sum_of_h = None
    # Compute the multiply factors
    alpha_list = np.log(1/self.beta_list)
    # For each hypothesis
    for index, h in enumerate(self.H):
      # Predict probabilities
      probabilities = h.predict_proba(input)
      # Multiply probabilities by the hypothesis computed alpha
      weighted_prob = alpha_list[index] * probabilities
      # If it's the first hypothesis
      if index == 0:
        sum_of_h = weighted_prob
      else:
        sum_of_h += weighted_prob
    # Choose the label using argmax
    predicted_idx = np.argmax(sum_of_h, axis=1)
    predicted_labels = []
    for pr in predicted_idx:
      predicted_labels.append(self.index_to_label[pr])
    predicted_labels = np.array(predicted_labels)
    #print(predicted_labels)
    # Remember the h_finite value
    h_fin = [sum_of_h[index][label]*predicted_labels[index] for index, label in enumerate(predicted_idx)]
    #print(h_fin)
    alpha_norm = np.linalg.norm(alpha_list, ord=1)
    h_fin = h_fin/alpha_norm
    return predicted_labels, h_fin

  # Print the computed pseudo-loss of each classifier
  def pseudo_losses(self):
    return self.loss_list


The final loss of the ensemble will be computed using a exponential loss function. It will only suitable for binary classification (-1 and +1 labels). The exponential loss function is sensitive to outliers/label noise.

$Loss(X) = \frac{1}{m}\sum_{i=1}^{m}e^{-y_i\bar{h}(x_i)}$, where $\bar{h}(x_i)=y\sum_{t=1}^{T}\alpha_th_t(x_i)$ for $y$ that maximizes the $h_{fin}(x_i)$.

### 2.2) Run on 5-fold

Train the algorithm using 4-fold data and evaluate its performance using the remaining 1-fold data.

In [24]:
# Remove last results
five_fold_results = five_fold_results[five_fold_results['Algorithm'] != 'AdaBoostM2']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  # Fold counter
  fold_count = 1
  # A list containing all the fold results
  all_fold_results = []
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # For each fold
  for train_index, test_index in five_fold.split(X):
    # Split the train and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Create an AdaBoostM2 object
    adaboostm2 = AdaBoostM2(X_train, y_train, t)
    # Run the algorithm
    adaboostm2.run()
    # Add scatter of this pseudo-loss to the loss plotter
    pseudo_losses = adaboostm2.pseudo_losses()
    loss_plotter.add_scatter(pseudo_losses, 'fold-'+str(fold_count))
    # Prediction on the test data
    predictions, hfin_values = adaboostm2.predict(X_test)
    model_margin = abs(hfin_values[0])
    for value in hfin_values:
      current_margin = abs(value)
      if current_margin < model_margin:
        model_margin = current_margin
    # Plot ROC curve
    auc_score = roc_plotter.add_scatter(y_test, hfin_values, 'Fold-'+str(fold_count))
    # Compute the loss on the test data
    loss = ensemble_loss_binary(y_test, predictions, hfin_values)
    results = {'Loss': loss, 'Margin': model_margin}
    results['AUC'] = auc_score
    results.update(get_metrics(y_test, predictions))
    all_fold_results.append(results)
    print('Fold-'+str(fold_count), results, '\n')
    fold_count += 1
  
  # Compute the average
  df_results = pd.DataFrame(all_fold_results)
  avg_results = dict(df_results.mean())
  # Print a horizontal line
  print ('-' * 128)

  # Remember the results
  avg_results['Algorithm'] = 'AdaBoostM2'
  avg_results['T'] = t
  five_fold_results = five_fold_results.append(avg_results, ignore_index=True)
  print('Average', avg_results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()

T= 10
--------------------------------------------------------------------------------------------------------------------------------



Fold-1 {'Loss': 0.8559433439186134, 'Margin': 0.5514066609695649, 'AUC': 0.6952872455055861, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7483660130718954, 'TN': 229, 'FP': 0, 'FN': 77, 'TP': 0} 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Fold-2 {'Loss': 0.8433981899995954, 'Margin': 0.5824196795699598, 'AUC': 0.6234934446469517, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.761437908496732, 'TN': 233, 'FP': 0, 'FN': 73, 'TP': 0} 



...


Fold-1 {'Loss': 0.8392833481040796, 'Margin': 0.5004962583962422, 'AUC': 0.7288039471445585, 'Precision': 1.0, 'Recall': 0.05194805194805195, 'F1score': 0.09876543209876544, 'Accuracy': 0.761437908496732, 'TN': 229, 'FP': 0, 'FN': 73, 'TP': 4} 




Fold-2 {'Loss': 0.8421295672627546, 'Margin': 0.5012715835654572, 'AUC': 0.7191780821917808, 'Precision': 0.4838709677419355, 'Recall': 0.2054794520547945, 'F1score': 0.28846153846153844, 'Accuracy': 0.7581699346405228, 'TN': 217, 'FP': 16, 'FN': 58, 'TP': 15} 




Fold-3 {'Loss': 0.8111085158586379, 'Margin': 0.5007888587331137, 'AUC': 0.7913698955365622, 'Precision': 1.0, 'Recall': 0.06944444444444445, 'F1score': 0.12987012987012989, 'Accuracy': 0.7810457516339869, 'TN': 234, 'FP': 0, 'FN': 67, 'TP': 5} 




Fold-4 {'Loss': 0.8798106769014964, 'Margin': 0.5044202962560869, 'AUC': 0.7342463625537203, 'Precision': 0.875, 'Recall': 0.07865168539325842, 'F1score': 0.1443298969072165, 'Accuracy': 0.7287581699346405, 'TN': 216, 'FP': 1, 'FN': 82, 'TP': 7} 




Fold-5 {'Loss': 0.9017000849077924, 'Margin': 0.5068022471431224, 'AUC': 0.7485196708149336, 'Precision': 0.7272727272727273, 'Recall': 0.0851063829787234, 'F1score': 0.15238095238095237, 'Accuracy': 0.7091503267973857, 'TN': 209, 'FP': 3, 'FN': 86, 'TP': 8} 

--------------------------------------------------------------------------------------------------------------------------------
Average {'Loss': 0.8548064386069523, 'Margin': 0.5027558488188045, 'AUC': 0.7444235916483112, 'Precision': 0.8172287390029325, 'Recall': 0.09812600336385455, 'F1score': 0.16276158994372053, 'Accuracy': 0.7477124183006536, 'TN': 221.0, 'FP': 4.0, 'FN': 73.2, 'TP': 7.8, 'Algorithm': 'AdaBoostM2', 'T': 100}


### 2.3) Run on whole training data

Train and evaluate the algorithm both on the training data.

In [25]:
# Remove last results
train_results = train_results[train_results['Algorithm'] != 'AdaBoostM2']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # Create an AdaBoostM2 object
  adaboostm2 = AdaBoostM2(X, y, t)
  # Run the algorithm
  adaboostm2.run()
  # Add scatter of this pseudo-loss to the loss plotter
  pseudo_losses = adaboostm2.pseudo_losses()
  loss_plotter.add_scatter(pseudo_losses, 'train data')
  # Prediction on the test data
  predictions, hfin_values = adaboostm2.predict(X)
  model_margin = abs(hfin_values[0])
  for value in hfin_values:
    current_margin = abs(value)
    if current_margin < model_margin:
      model_margin = current_margin
  # Plot ROC curve
  auc_score = roc_plotter.add_scatter(y, hfin_values, 'train data')
  # Compute the loss on the test data
  loss = ensemble_loss_binary(y, predictions, hfin_values)
  # Compute metrics
  results = {'Loss': loss, 'Margin': model_margin}
  results.update(get_metrics(y, predictions))

  # Remember the results
  results['Algorithm'] = 'AdaBoostM2'
  results['T'] = t
  results['AUC'] = auc_score
  train_results = train_results.append(results, ignore_index=True)
  print(results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()
  plot_histogram(y, hfin_values, 'The histogram of scores compared to true labels for T='+str(t))

T= 10
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8678048200433963, 'Margin': 0.5316338751574275, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 1125, 'FP': 0, 'FN': 405, 'TP': 0, 'Algorithm': 'AdaBoostM2', 'T': 10, 'AUC': 0.7326748971193415}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



T= 50
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8663441215300979, 'Margin': 0.5034791452583326, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 1125, 'FP': 0, 'FN': 405, 'TP': 0, 'Algorithm': 'AdaBoostM2', 'T': 50, 'AUC': 0.7518244170096021}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



T= 100
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8460690220661642, 'Margin': 0.5057917274255974, 'Precision': 0.875, 'Recall': 0.08641975308641975, 'F1score': 0.15730337078651688, 'Accuracy': 0.7549019607843137, 'TN': 1120, 'FP': 5, 'FN': 370, 'TP': 35, 'Algorithm': 'AdaBoostM2', 'T': 100, 'AUC': 0.7593415637860083}


## 3) RUSBoost

**RUSBoost** provides a simpler and faster alternative to *SMOTEBoost*. The way it works is so similar to *AdaBoost.M2*. The only difference is in each iteration (training a new hypothesis), Random Under Sampling (RUS) is applied to remove the majority class examples until $N\%$ of the temporary training data set $S_t'$ belongs to the minority class [[5]](https://ieeexplore.ieee.org/document/5299216).

![picture](https://drive.google.com/uc?id=1KD6uEISQzB5eoEuac1JfuRZhKizza_Yq)

### 3.1) Algorithm implementation

In [26]:
class RUSBoost:
  """
    Functions:
      run(self) -> Run the algorithm of RUSBoost
      RUS(self, D_t) -> Choose the samples using the Random Undersampling technique
      predict(self, input) -> Predict the labels for given samples using created ensemble
      pseudo_losses(self) -> The pseudo-loss for each hypothesis
  """


  # Initilizition
  def __init__(self, X, y, T, N):
    """
      X: Feature vectors
      y: Labels
      m: Number of samples
      T: Number of iterations/classifiers/hypothesis
      N: The N% of the new training set in RUS belongs to the minority class
      H: Ensemble of classifiers
      beta_list: A list containing the beta value for each classifier
      loss_list: A list containing the pseudo-loss of each classifier
    """
    self.X = X
    self.y = y
    self.m = len(X)
    self.T = T
    self.N = N
    self.H = []
    self.beta_list = []
    self.loss_list = []
    self.minority_label = 0
    self.majority_label = 0
    self.minority_size = 0
    self.majority_size = 0
    self.minority_idx = []
    self.majority_idx = []
    self.label_to_index = {}
    self.index_to_label = {}
    self.set_minority_majority()

  # Find which label is minority and which majority
  def set_minority_majority(self):
    labels = np.unique(self.y)
    count = []
    for label in labels:
      cnt = len((self.y==label).nonzero()[0])
      count.append(cnt)
    minority_idx = 0
    majority_idx = 0
    if count[0]>count[1]:
      minority_idx = 1
    else:
      majority_idx = 1
    self.majority_label = labels[majority_idx]
    self.majority_size = count[majority_idx]
    self.majority_idx = (self.y == labels[majority_idx]).nonzero()[0]
    self.minority_label = labels[minority_idx]
    self.minority_size = count[minority_idx]
    self.minority_idx = (self.y == labels[minority_idx]).nonzero()[0]

  # Choose the samples using RUS
  def RUS(self, D_t):
    needed_majority = round(((100-self.N) * self.minority_size)/self.N)
    # Make sure that the maximum possible number for needed data 
    # is the number of data in majority class 
    if needed_majority > self.majority_size:
      needed_majority = self.majority_size
    # Randomly choose the needed number unique data from majority class
    chosen_idx = np.random.choice(self.majority_idx, needed_majority, replace=False)
    # Make the data ready
    chosen_majority = self.X[chosen_idx]
    minority_data = self.X[self.minority_idx]
    # Concatenate the randomly chosen majority data with minority data
    data = np.concatenate((chosen_majority, minority_data), axis=0)
    # Make the weights ready
    chosen_weight = D_t[chosen_idx]
    minority_weight = D_t[self.minority_idx]
    # Concatenate the randomly chosen majority data with minority data
    weight = np.concatenate((chosen_weight, minority_weight), axis=0)
    # Make the label lists
    label_majority = [self.majority_label for i in chosen_majority]
    label_minority = [self.minority_label for i in minority_data]
    label = label_majority + label_minority
    label = np.array(label)
    # Shuffle the data
    data, label, weight = shuffle(data, label, weight)
    return data, label, weight
  

  # Running the RUSBoost algorithm
  def run(self):

    # Initilize distribution
    D_t = (1./self.m) * np.ones(self.m)

    for t in trange(self.T):

      # Choose the new (temporary) training set
      X_temp, y_temp, D_temp = self.RUS(D_t)

      # Train a new classifier on training data with distibution D_t
      weak_learn = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
      weak_learn.fit(X_temp, y_temp, sample_weight=D_temp)
      
      # Get predictions and probabilities for the training data
      predictions = weak_learn.predict(self.X)
      probabilities = weak_learn.predict_proba(self.X)
      
      if t == 0:
        for index, value in enumerate(weak_learn.classes_):
          self.index_to_label[index] = value
          self.label_to_index[value] = index
      
      # Compute pseudo-loss
      pseudo_loss = 0

      # For each sample
      for index, true_label in enumerate(self.y):
        
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]

        # For each wrong label
        current_loss = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            current_loss += D_t[index] * (1 - true_prob + wrong_prob)
        # Add it to pseudo_loss
        pseudo_loss += current_loss

      # Final step to compute pseudo-loss
      pseudo_loss = (1/2)*pseudo_loss

      # Compute beta
      beta = pseudo_loss / (1 - pseudo_loss)
      
      # Update distribution
      for index, value in enumerate(D_t):
        predicted_label = predictions[index]
        true_label = self.y[index]
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]
        # Probability of assigning the wrong label
        power = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            power += (1 + true_prob - wrong_prob)
        power = (1/2) * power
        D_t[index] = (value) * pow(beta,power)
      
      # Normalization factor
      Z_t = np.sum(D_t)
      D_t = D_t / Z_t
      
      # Append weak learner to the ensemble
      self.H.append(weak_learn)
      # Append computed loss of the weak learner
      self.loss_list.append(pseudo_loss)
      # Append computed beta of the weak learner
      self.beta_list.append(beta)
  
      # print('t =', t, '--> pseudo-loss =', pseudo_loss)
    
    # Convert beta and loss lists to numpy array
    self.beta_list = np.array(self.beta_list)
    self.loss_list = np.array(self.loss_list)


  def predict(self, input):
    # Sum of the probabilities made by each hypothesis for a given sample and label
    sum_of_h = None
    # Compute the multiply factors
    alpha_list = np.log(1/self.beta_list)
    # For each hypothesis
    for index, h in enumerate(self.H):
      # Predict probabilities
      probabilities = h.predict_proba(input)
      # Multiply probabilities by the hypothesis computed alpha
      weighted_prob = alpha_list[index] * probabilities
      # If it's the first hypothesis
      if index == 0:
        sum_of_h = weighted_prob
      else:
        sum_of_h += weighted_prob
    # Choose the label using argmax
    predicted_idx = np.argmax(sum_of_h, axis=1)
    predicted_labels = []
    for pr in predicted_idx:
      predicted_labels.append(self.index_to_label[pr])
    predicted_labels = np.array(predicted_labels)
    #print(predicted_labels)
    # Remember the h_finite value
    h_fin = [sum_of_h[index][label]*predicted_labels[index] for index, label in enumerate(predicted_idx)]
    #print(h_fin)
    alpha_norm = np.linalg.norm(alpha_list, ord=1)
    h_fin = h_fin/alpha_norm
    return predicted_labels, h_fin

  # Print the computed pseudo-loss of each classifier
  def pseudo_losses(self):
    return self.loss_list


### 3.2) Run on 5-fold

In [27]:
# Remove last results
five_fold_results = five_fold_results[five_fold_results['Algorithm'] != 'RUSBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  # Fold counter
  fold_count = 1
  # A list containing all the fold results
  all_fold_results = []
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # For each fold
  for train_index, test_index in five_fold.split(X):
    # Split the train and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Create an RUSBoost object
    rusboost = RUSBoost(X_train, y_train, t, 50)
    # Run the algorithm
    rusboost.run()
    # Add scatter of this pseudo-loss to the loss plotter
    pseudo_losses = rusboost.pseudo_losses()
    loss_plotter.add_scatter(pseudo_losses, 'fold-'+str(fold_count))
    # Prediction on the test data
    predictions, hfin_values = rusboost.predict(X_test)
    model_margin = abs(hfin_values[0])
    for value in hfin_values:
      current_margin = abs(value)
      if current_margin < model_margin:
        model_margin = current_margin
    # Plot ROC curve
    auc_score = roc_plotter.add_scatter(y_test, hfin_values, 'Fold-'+str(fold_count))
    # Compute the loss on the test data
    loss = ensemble_loss_binary(y_test, predictions, hfin_values)
    results = {'Loss': loss, 'Margin': model_margin}
    results['AUC'] = auc_score
    results.update(get_metrics(y_test, predictions))
    all_fold_results.append(results)
    print('Fold-'+str(fold_count), results, '\n')
    fold_count += 1
  
  # Compute the average
  df_results = pd.DataFrame(all_fold_results)
  avg_results = dict(df_results.mean())
  # Print a horizontal line
  print ('-' * 128)

  # Remember the results
  avg_results['Algorithm'] = 'RUSBoost'
  avg_results['T'] = t
  five_fold_results = five_fold_results.append(avg_results, ignore_index=True)
  print('Average', avg_results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()

T= 10
--------------------------------------------------------------------------------------------------------------------------------



Fold-1 {'Loss': 0.9232128885131777, 'Margin': 0.5448182057600143, 'AUC': 0.6719786763454887, 'Precision': 0.43010752688172044, 'Recall': 0.5194805194805194, 'F1score': 0.4705882352941176, 'Accuracy': 0.7058823529411765, 'TN': 176, 'FP': 53, 'FN': 37, 'TP': 40} 




Fold-2 {'Loss': 0.8936791881607702, 'Margin': 0.5141408893999035, 'AUC': 0.7170615556470104, 'Precision': 0.43478260869565216, 'Recall': 0.410958904109589, 'F1score': 0.4225352112676056, 'Accuracy': 0.7320261437908496, 'TN': 194, 'FP': 39, 'FN': 43, 'TP': 30} 




Fold-3 {'Loss': 0.9083350640726638, 'Margin': 0.5087598798620017, 'AUC': 0.7394349477682811, 'Precision': 0.4336283185840708, 'Recall': 0.6805555555555556, 'F1score': 0.5297297297297296, 'Accuracy': 0.7156862745098039, 'TN': 170, 'FP': 64, 'FN': 23, 'TP': 49} 




Fold-4 {'Loss': 0.9728953055892925, 'Margin': 0.5337703841919987, 'AUC': 0.6756847719152902, 'Precision': 0.44642857142857145, 'Recall': 0.5617977528089888, 'F1score': 0.49751243781094534, 'Accuracy': 0.6699346405228758, 'TN': 155, 'FP': 62, 'FN': 39, 'TP': 50} 



...


Fold-1 {'Loss': 0.9272809745849337, 'Margin': 0.5030576590229869, 'AUC': 0.7318947428117734, 'Precision': 0.4158415841584158, 'Recall': 0.5454545454545454, 'F1score': 0.47191011235955055, 'Accuracy': 0.6928104575163399, 'TN': 170, 'FP': 59, 'FN': 35, 'TP': 42} 




Fold-2 {'Loss': 0.8833627930069093, 'Margin': 0.5008144861050873, 'AUC': 0.7277911693809159, 'Precision': 0.4411764705882353, 'Recall': 0.410958904109589, 'F1score': 0.42553191489361697, 'Accuracy': 0.7352941176470589, 'TN': 195, 'FP': 38, 'FN': 43, 'TP': 30} 




Fold-3 {'Loss': 0.8686314980470499, 'Margin': 0.5016083544108364, 'AUC': 0.7867105887939221, 'Precision': 0.4716981132075472, 'Recall': 0.6944444444444444, 'F1score': 0.5617977528089889, 'Accuracy': 0.7450980392156863, 'TN': 178, 'FP': 56, 'FN': 22, 'TP': 50} 




Fold-4 {'Loss': 0.9344804753985072, 'Margin': 0.5013769487284041, 'AUC': 0.7299228498938539, 'Precision': 0.47368421052631576, 'Recall': 0.6067415730337079, 'F1score': 0.5320197044334976, 'Accuracy': 0.6895424836601307, 'TN': 157, 'FP': 60, 'FN': 35, 'TP': 54} 




Fold-5 {'Loss': 0.9008593599792982, 'Margin': 0.5026688153302749, 'AUC': 0.7475411481332799, 'Precision': 0.5338983050847458, 'Recall': 0.6702127659574468, 'F1score': 0.5943396226415094, 'Accuracy': 0.7189542483660131, 'TN': 157, 'FP': 55, 'FN': 31, 'TP': 63} 

--------------------------------------------------------------------------------------------------------------------------------
Average {'Loss': 0.9029230202033398, 'Margin': 0.5019052527195179, 'AUC': 0.744772099802749, 'Precision': 0.46725973671305193, 'Recall': 0.5855624465999467, 'F1score': 0.5171198214274326, 'Accuracy': 0.7163398692810459, 'TN': 171.4, 'FP': 53.6, 'FN': 33.2, 'TP': 47.8, 'Algorithm': 'RUSBoost', 'T': 100}


### 3.3) Run on whole training data

In [28]:
# Remove last results
train_results = train_results[train_results['Algorithm'] != 'RUSBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # Create an RUSBoost object
  rusboost = RUSBoost(X, y, t, 50)
  # Run the algorithm
  rusboost.run()
  # Add scatter of this pseudo-loss to the loss plotter
  pseudo_losses = rusboost.pseudo_losses()
  loss_plotter.add_scatter(pseudo_losses, 'train data')
  # Prediction on the test data
  predictions, hfin_values = rusboost.predict(X)
  model_margin = abs(hfin_values[0])
  for value in hfin_values:
    current_margin = abs(value)
    if current_margin < model_margin:
      model_margin = current_margin
  # Plot ROC curve
  auc_score = roc_plotter.add_scatter(y, hfin_values, 'train data')
  # Compute the loss on the test data
  loss = ensemble_loss_binary(y, predictions, hfin_values)
  # Compute metrics
  results = {'Loss': loss, 'Margin': model_margin}
  results.update(get_metrics(y, predictions))

  # Remember the results
  results['Algorithm'] = 'RUSBoost'
  results['T'] = t
  results['AUC'] = auc_score
  train_results = train_results.append(results, ignore_index=True)
  print(results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()
  plot_histogram(y, hfin_values, 'The histogram of scores compared to true labels for T='+str(t))

T= 10
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.933030729745024, 'Margin': 0.5069887898361252, 'Precision': 0.4462809917355372, 'Recall': 0.6666666666666666, 'F1score': 0.5346534653465347, 'Accuracy': 0.6928104575163399, 'TN': 790, 'FP': 335, 'FN': 135, 'TP': 270, 'Algorithm': 'RUSBoost', 'T': 10, 'AUC': 0.7265569272976682}


T= 50
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.9034996636813292, 'Margin': 0.5036749581500902, 'Precision': 0.4732142857142857, 'Recall': 0.654320987654321, 'F1score': 0.5492227979274611, 'Accuracy': 0.7156862745098039, 'TN': 830, 'FP': 295, 'FN': 140, 'TP': 265, 'Algorithm': 'RUSBoost', 'T': 50, 'AUC': 0.7474348422496571}


T= 100
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8918991536023327, 'Margin': 0.5036975972748579, 'Precision': 0.4854368932038835, 'Recall': 0.6172839506172839, 'F1score': 0.5434782608695653, 'Accuracy': 0.7254901960784313, 'TN': 860, 'FP': 265, 'FN': 155, 'TP': 250, 'Algorithm': 'RUSBoost', 'T': 100, 'AUC': 0.7606584362139918}


## 4) SMOTEBoost

SMOTEBoost goal is to utilize SMOTE [[6]](https://arxiv.org/pdf/1106.1813.pdf) for improving the prediction of the minority classes, and to utilize boosting to not sacrifice accuracy over the entire data set [[7]](https://link.springer.com/chapter/10.1007/978-3-540-39804-2_12). The new synthetic minority samples are created as follows :
*  Take the difference between a feature vector (minority class sample) and one of its k nearest neighbors (minority class samples).
*  Multiply this difference by a random number between 0 and 1.
*  Add this difference to the feature value of the original feature vector, thus creating a new feature vector.

### 4.1) K-Nearest Neighbors

The following code is a pure implementation of finding k-nearest neighbors. As expected it's not time-efficient, so it's better to use a better implementation from a well-known library like sklearn [[8]](https://scikit-learn.org/stable/modules/neighbors.html).

In [29]:
def KNN(samples, chosen, k):
  distances = []
  for index, sample in enumerate(samples):
    # np.linalg.norm computes the euclidean distance
    dist = np.linalg.norm(chosen - sample)
    distances.append((index, dist))
    distances.sort(key=lambda tup: tup[1])
  # A list containing the k neares neighbors
  k_neighbors = []
  index = 0
  created_num = 0
  while (created_num != k):
    sample_index = distances[index][0]
    neighbor = samples[sample_index]
    # The chosen sample itself (distance=0) should not be considered
    if (distances[index][1] != 0):
      k_neighbors.append(neighbor)
      created_num += 1
    index += 1
  return k_neighbors

### 4.2) SMOTE

The **SMOTE** function creates synthetic examples based on given samples.   
Consider that $\alpha = \frac{amount}{100}$ the function looks at each given sample and their $k$ neighbors, then creates $int(\alpha)$ synthetic examples based on each of them. Also for the floating part of the $\alpha$, it randomly chooses samples and creates one synthetic example per each of them.

In [30]:
def SMOTE(given_samples, amount, k):
  # The number of samples
  samples_num = len(given_samples)
  # The total number of samples need to be created
  needed_num = (amount/100)*samples_num
  # The number of synthetic samples need to be created from each sample
  full_num = int(needed_num/samples_num)
  # The number of samples need to be created from random samples
  remain_num = round(needed_num - (full_num*samples_num))
  # A list containing the new synthetic samples
  synthetic_samples = []
  # The defulat metric is euclidean distance
  nbrs = NearestNeighbors(n_neighbors=k).fit(given_samples)

  for sample in given_samples:
    # Find k nearest neighbors
    #k_neighbors = KNN(given_samples, sample, k)
    distances, indices = nbrs.kneighbors(sample.reshape(1, -1))
    non_zero_indices = []
    for i, d in enumerate(distances[0]):
      if d != 0:
        non_zero_indices.append(indices[0][i])
    # Choose 'full_num' random samples from this neighbors
    chosen_idx = np.random.choice(non_zero_indices, full_num, replace=False)
    for idx in chosen_idx:
      chosen = given_samples[idx]
      difference = abs(sample - chosen)
      rand_num = np.random.random(1)
      created_sample = sample + (rand_num*difference)
      synthetic_samples.append(created_sample)

  # For the ramaining number, we choose randomly from given_samples
  # and create just one synthetic sample from each of them
  chosen_idx = np.random.choice(len(given_samples), remain_num, replace=False)
  for idx in chosen_idx:
    non_zero_indices = []
    for i, d in enumerate(distances[0]):
      if d != 0:
        non_zero_indices.append(indices[0][i])
    chosen_neighbor_idx = np.random.choice(non_zero_indices, 1)
    chosen_neighbor = given_samples[chosen_neighbor_idx[0]]
    difference = given_samples[idx] - chosen_neighbor
    rand_num = np.random.random(1)
    created_sample = given_samples[idx] + (rand_num*difference)
    synthetic_samples.append(created_sample)
  
  # Shuffle the data
  return np.array(synthetic_samples)

### 4.3) Algorithm implementation

The procedure of **SMOTEBoost** algorithm consists of introducing SMOTE in each round of boosting.   
Therefore each learner will be able to learn from more of the minority class cases, thus learning broader decision regions for the minority class.

<font color='red'>**Note:**</font> The synthetically created minority class cases are discarded after learning a classifier at each iteration. That is, they are not added to the original data set.

![picture](https://drive.google.com/uc?id=1_n6gZPKL2MaD9CvPBeJctFkaRVUuI3gK)

In [31]:
class SMOTEBoost:
  """
    Functions:
      run(self) -> Run the algorithm of SMOTEBoost
      predict(self, input) -> Predict the labels for given samples using created ensemble
      pseudo_losses(self) -> The pseudo-loss for each hypothesis
  """


  # Initilizition
  def __init__(self, X, y, T, N, k):
    """
      X: Feature vectors
      y: Labels
      m: Number of samples
      T: Number of iterations/classifiers/hypothesis
      N: The SMOTE amount (percentage)
      k: The value of k in KNN when introducing SMOTE
      H: Ensemble of classifiers
      beta_list: A list containing the beta value for each classifier
      loss_list: A list containing the pseudo-loss of each classifier
    """
    self.X = X
    self.y = y
    self.m = len(X)
    self.T = T
    self.N = N
    self.k = k
    self.H = []
    self.beta_list = []
    self.loss_list = []
    self.minority_label = 0
    self.majority_label = 0
    self.minority_size = 0
    self.majority_size = 0
    self.minority_data = []
    self.majority_data = []
    self.index_to_label = {}
    self.label_to_index = {}
    self.set_minority_majority()

  # Find which label is minority and which majority
  def set_minority_majority(self):
    labels = np.unique(self.y)
    count = []
    for label in labels:
      cnt = len((self.y==label).nonzero()[0])
      count.append(cnt)
    minority_idx = 0
    majority_idx = 0
    if count[0]>count[1]:
      minority_idx = 1
    else:
      majority_idx = 1
    self.majority_label = labels[majority_idx]
    self.majority_size = count[majority_idx]
    self.majority_data = self.X[(self.y == labels[majority_idx]).nonzero()[0]]
    self.minority_label = labels[minority_idx]
    self.minority_size = count[minority_idx]
    self.minority_data = self.X[(self.y == labels[minority_idx]).nonzero()[0]]

  # Running the SMOTEBoost algorithm
  def run(self):

    # Initilize distribution
    D_t = (1./self.m) * np.ones(self.m)

    for t in trange(self.T):
      X_synthetic = SMOTE(self.minority_data, self.N, self.k)
      y_synthetic = np.full(X_synthetic.shape[0], dtype=np.int64,
                           fill_value=self.minority_label)
      D_synthetic = np.full(X_synthetic.shape[0], fill_value=(1./self.m))
      X_new = np.vstack((self.X, X_synthetic))
      y_new = np.concatenate((self.y, y_synthetic))
      D_new = np.concatenate((D_t, D_synthetic))
      

      # Train a new classifier on training data with distibution D_t
      weak_learn = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
      weak_learn.fit(X_new, y_new, sample_weight=D_new)
      
      # Get predictions and probabilities for the training data
      predictions = weak_learn.predict(self.X)
      probabilities = weak_learn.predict_proba(self.X)
      
      if t == 0:
        for index, value in enumerate(weak_learn.classes_):
          self.index_to_label[index] = value
          self.label_to_index[value] = index
      
      # Compute pseudo-loss
      pseudo_loss = 0

      # For each sample
      for index, true_label in enumerate(self.y):
        
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]

        # For each wrong label
        current_loss = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            current_loss += D_t[index] * (1 - true_prob + wrong_prob)
        # Add it to pseudo_loss
        pseudo_loss += current_loss

      # Final step to compute pseudo-loss
      pseudo_loss = (1/2)*pseudo_loss

      # Compute beta
      beta = pseudo_loss / (1 - pseudo_loss)
      
      # Update distribution
      for index, value in enumerate(D_t):
        predicted_label = predictions[index]
        true_label = self.y[index]
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]
        # Probability of assigning the wrong label
        power = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            power += (1 + true_prob - wrong_prob)
        power = (1/2) * power
        D_t[index] = (value) * pow(beta,power)
      
      # Normalization factor
      Z_t = np.sum(D_t)
      D_t = D_t / Z_t
      
      # Append weak learner to the ensemble
      self.H.append(weak_learn)
      # Append computed loss of the weak learner
      self.loss_list.append(pseudo_loss)
      # Append computed beta of the weak learner
      self.beta_list.append(beta)
  
      # print('t =', t, '--> pseudo-loss =', pseudo_loss)
    
    # Convert beta and loss lists to numpy array
    self.beta_list = np.array(self.beta_list)
    self.loss_list = np.array(self.loss_list)


  def predict(self, input):
    # Sum of the probabilities made by each hypothesis for a given sample and label
    sum_of_h = None
    # Compute the multiply factors
    alpha_list = np.log(1/self.beta_list)
    # For each hypothesis
    for index, h in enumerate(self.H):
      # Predict probabilities
      probabilities = h.predict_proba(input)
      # Multiply probabilities by the hypothesis computed alpha
      weighted_prob = alpha_list[index] * probabilities
      # If it's the first hypothesis
      if index == 0:
        sum_of_h = weighted_prob
      else:
        sum_of_h += weighted_prob
    # Choose the label using argmax
    predicted_idx = np.argmax(sum_of_h, axis=1)
    predicted_labels = []
    for pr in predicted_idx:
      predicted_labels.append(self.index_to_label[pr])
    predicted_labels = np.array(predicted_labels)
    #print(predicted_labels)
    # Remember the h_finite value
    h_fin = [sum_of_h[index][label]*predicted_labels[index] for index, label in enumerate(predicted_idx)]
    #print(h_fin)
    alpha_norm = np.linalg.norm(alpha_list, ord=1)
    h_fin = h_fin/alpha_norm
    return predicted_labels, h_fin

  # Print the computed pseudo-loss of each classifier
  def pseudo_losses(self):
    return self.loss_list


### 4.4) Run on 5-fold

In [32]:
# Remove last results
five_fold_results = five_fold_results[five_fold_results['Algorithm'] != 'SMOTEBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  # Fold counter
  fold_count = 1
  # A list containing all the fold results
  all_fold_results = []
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # For each fold
  for train_index, test_index in five_fold.split(X):
    # Split the train and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Create an SMOTEBoost object
    smoteboost = SMOTEBoost(X_train, y_train, t, smote_ratio, k_in_knn)
    # Run the algorithm
    smoteboost.run()
    # Add scatter of this pseudo-loss to the loss plotter
    pseudo_losses = smoteboost.pseudo_losses()
    loss_plotter.add_scatter(pseudo_losses, 'fold-'+str(fold_count))
    # Prediction on the test data
    predictions, hfin_values = smoteboost.predict(X_test)
    model_margin = abs(hfin_values[0])
    for value in hfin_values:
      current_margin = abs(value)
      if current_margin < model_margin:
        model_margin = current_margin
    # Plot ROC curve
    auc_score = roc_plotter.add_scatter(y_test, hfin_values, 'Fold-'+str(fold_count))
    # Compute the loss on the test data
    loss = ensemble_loss_binary(y_test, predictions, hfin_values)
    results = {'Loss': loss, 'Margin': model_margin}
    results['AUC'] = auc_score
    results.update(get_metrics(y_test, predictions))
    all_fold_results.append(results)
    print('Fold-'+str(fold_count), results, '\n')
    fold_count += 1
  
  # Compute the average
  df_results = pd.DataFrame(all_fold_results)
  avg_results = dict(df_results.mean())
  # Print a horizontal line
  print ('-' * 128)

  # Remember the results
  avg_results['Algorithm'] = 'SMOTEBoost'
  avg_results['T'] = t
  five_fold_results = five_fold_results.append(avg_results, ignore_index=True)
  print('Average', avg_results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()

T= 10
--------------------------------------------------------------------------------------------------------------------------------



Fold-1 {'Loss': 0.8710888413622714, 'Margin': 0.5067024838777824, 'AUC': 0.6625928656496342, 'Precision': 0.49382716049382713, 'Recall': 0.5194805194805194, 'F1score': 0.5063291139240506, 'Accuracy': 0.7450980392156863, 'TN': 188, 'FP': 41, 'FN': 37, 'TP': 40} 




Fold-2 {'Loss': 0.8707927024202595, 'Margin': 0.534084629649702, 'AUC': 0.6495972720324533, 'Precision': 0.45614035087719296, 'Recall': 0.3561643835616438, 'F1score': 0.4, 'Accuracy': 0.7450980392156863, 'TN': 202, 'FP': 31, 'FN': 47, 'TP': 26} 




Fold-3 {'Loss': 0.8230247762625382, 'Margin': 0.5084511949484297, 'AUC': 0.7397317188983855, 'Precision': 0.5394736842105263, 'Recall': 0.5694444444444444, 'F1score': 0.5540540540540541, 'Accuracy': 0.7843137254901961, 'TN': 199, 'FP': 35, 'FN': 31, 'TP': 41} 




Fold-4 {'Loss': 0.9053216087730429, 'Margin': 0.5012224376392843, 'AUC': 0.6791539377621291, 'Precision': 0.5194805194805194, 'Recall': 0.449438202247191, 'F1score': 0.48192771084337344, 'Accuracy': 0.7189542483660131, 'TN': 180, 'FP': 37, 'FN': 49, 'TP': 40} 



...


Fold-1 {'Loss': 0.8683724144089046, 'Margin': 0.506182001070279, 'AUC': 0.6788691657687291, 'Precision': 0.49295774647887325, 'Recall': 0.45454545454545453, 'F1score': 0.47297297297297297, 'Accuracy': 0.7450980392156863, 'TN': 193, 'FP': 36, 'FN': 42, 'TP': 35} 




Fold-2 {'Loss': 0.8653843689736234, 'Margin': 0.5027482038695986, 'AUC': 0.7135928038097477, 'Precision': 0.46551724137931033, 'Recall': 0.3698630136986301, 'F1score': 0.4122137404580153, 'Accuracy': 0.7483660130718954, 'TN': 202, 'FP': 31, 'FN': 46, 'TP': 27} 




Fold-3 {'Loss': 0.7920042749948116, 'Margin': 0.5002072676154211, 'AUC': 0.7631172839506173, 'Precision': 0.6153846153846154, 'Recall': 0.5555555555555556, 'F1score': 0.583941605839416, 'Accuracy': 0.8137254901960784, 'TN': 209, 'FP': 25, 'FN': 32, 'TP': 40} 




Fold-4 {'Loss': 0.8678491363583937, 'Margin': 0.5044776331055236, 'AUC': 0.7042406669083001, 'Precision': 0.5797101449275363, 'Recall': 0.449438202247191, 'F1score': 0.5063291139240506, 'Accuracy': 0.7450980392156863, 'TN': 188, 'FP': 29, 'FN': 49, 'TP': 40} 




Fold-5 {'Loss': 0.8426703035167996, 'Margin': 0.5112223039423872, 'AUC': 0.742397631473304, 'Precision': 0.6493506493506493, 'Recall': 0.5319148936170213, 'F1score': 0.5847953216374269, 'Accuracy': 0.7679738562091504, 'TN': 185, 'FP': 27, 'FN': 44, 'TP': 50} 

--------------------------------------------------------------------------------------------------------------------------------
Average {'Loss': 0.8472560996505065, 'Margin': 0.504967481920642, 'AUC': 0.7204435103821396, 'Precision': 0.5605840795041969, 'Recall': 0.47226342393277054, 'F1score': 0.5120505509663763, 'Accuracy': 0.7640522875816993, 'TN': 195.4, 'FP': 29.6, 'FN': 42.6, 'TP': 38.4, 'Algorithm': 'SMOTEBoost', 'T': 100}


### 4.5) Run on whole training data

In [33]:
# Remove last results
train_results = train_results[train_results['Algorithm'] != 'SMOTEBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # Create an SMOTEBoost object
  smoteboost = SMOTEBoost(X, y, t, smote_ratio, k_in_knn)
  # Run the algorithm
  smoteboost.run()
  # Add scatter of this pseudo-loss to the loss plotter
  pseudo_losses = smoteboost.pseudo_losses()
  loss_plotter.add_scatter(pseudo_losses, 'train data')
  # Prediction on the test data
  predictions, hfin_values = smoteboost.predict(X)
  model_margin = abs(hfin_values[0])
  for value in hfin_values:
    current_margin = abs(value)
    if current_margin < model_margin:
      model_margin = current_margin
  # Plot ROC curve
  auc_score = roc_plotter.add_scatter(y, hfin_values, 'train data')
  # Compute the loss on the test data
  loss = ensemble_loss_binary(y, predictions, hfin_values)
  # Compute metrics
  results = {'Loss': loss, 'Margin': model_margin}
  results.update(get_metrics(y, predictions))

  # Remember the results
  results['Algorithm'] = 'SMOTEBoost'
  results['T'] = t
  results['AUC'] = auc_score
  train_results = train_results.append(results, ignore_index=True)
  print(results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()
  plot_histogram(y, hfin_values, 'The histogram of scores compared to true labels for T='+str(t))

T= 10
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8716411834224502, 'Margin': 0.5164592363795695, 'Precision': 0.5131578947368421, 'Recall': 0.48148148148148145, 'F1score': 0.49681528662420377, 'Accuracy': 0.7418300653594772, 'TN': 940, 'FP': 185, 'FN': 210, 'TP': 195, 'Algorithm': 'SMOTEBoost', 'T': 10, 'AUC': 0.7081755829903977}


T= 50
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8457501237487096, 'Margin': 0.5068980327916796, 'Precision': 0.5692307692307692, 'Recall': 0.4567901234567901, 'F1score': 0.5068493150684931, 'Accuracy': 0.7647058823529411, 'TN': 985, 'FP': 140, 'FN': 220, 'TP': 185, 'Algorithm': 'SMOTEBoost', 'T': 50, 'AUC': 0.7211796982167353}


T= 100
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8387412793691775, 'Margin': 0.5085781569792657, 'Precision': 0.582089552238806, 'Recall': 0.48148148148148145, 'F1score': 0.527027027027027, 'Accuracy': 0.7712418300653595, 'TN': 985, 'FP': 140, 'FN': 210, 'TP': 195, 'Algorithm': 'SMOTEBoost', 'T': 100, 'AUC': 0.7312757201646093}


## 5) RBBoost

**RBBoost** algorithm uses both *SMOTE* and *RUS* techniques. Each member of the Random Balance ensemble is trained with data sampled from the training set and augmented by artificial instances obtained using SMOTE [[9]](https://www.sciencedirect.com/science/article/abs/pii/S0950705115001720).   
First of all, we should randomly set the new size of majority and minority classes. Then *SMOTE* and *Random Undersampling* are used to respectively increase or reduce the size of classes to match the desired size.

![picture](https://drive.google.com/uc?id=1BcI9Wqghyei2UxnH7Muop_t0ECK0n0Ff)

### 5.1) Algorithm implementation

In [35]:
class RBBoost:
  """
    Functions:
      run(self) -> Run the algorithm of RBBoost
      RandomBalance(self, D_T, k) -> Make new set using Random Balance technique 
      predict(self, input) -> Predict the labels for given samples using created ensemble
      pseudo_losses(self) -> The pseudo-loss for each hypothesis
  """


  # Initilizition
  def __init__(self, X, y, T, k):
    """
      X: Feature vectors
      y: Labels
      m: Number of samples
      T: Number of iterations/classifiers/hypothesis
      k: The value of k in KNN when introducing SMOTE
      H: Ensemble of classifiers
      beta_list: A list containing the beta value for each classifier
      loss_list: A list containing the pseudo-loss of each classifier
    """
    self.X = X
    self.y = y
    self.m = len(X)
    self.T = T
    self.k = k
    self.H = []
    self.beta_list = []
    self.loss_list = []
    self.minority_label = 0
    self.majority_label = 0
    self.minority_size = 0
    self.majority_size = 0
    self.minority_idx = []
    self.majority_idx = []
    self.index_to_label = {}
    self.label_to_index = {}
    self.set_minority_majority()

  # Find which label is minority and which majority
  def set_minority_majority(self):
    labels = np.unique(self.y)
    count = []
    for label in labels:
      cnt = len((self.y==label).nonzero()[0])
      count.append(cnt)
    minority_idx = 0
    majority_idx = 0
    if count[0]>count[1]:
      minority_idx = 1
    else:
      majority_idx = 1
    self.majority_label = labels[majority_idx]
    self.majority_size = count[majority_idx]
    self.majority_idx = (self.y == labels[majority_idx]).nonzero()[0]
    self.minority_label = labels[minority_idx]
    self.minority_size = count[minority_idx]
    self.minority_idx = (self.y == labels[minority_idx]).nonzero()[0]

  # Choose the samples using Random Balance technique
  def RandomBalance(self, D_t):
    new_majority_size = np.random.randint(2, self.m - 2)
    new_minority_size = self.m - new_majority_size
    X_new = []
    y_new = []
    D_new = []
    if new_majority_size < self.majority_size:
      # Add all minority samples to the new dataset
      minority_data = self.X[self.minority_idx]
      label_minority = [self.minority_label for i in minority_data]
      minority_weights = D_t[self.minority_idx]
      X_new.extend(minority_data)
      y_new.extend(label_minority)
      D_new.extend(minority_weights)
      # Take a random sample of size new_majority_size from majority class
      # and add the sample to the new dataset
      rand_majority_idx = np.random.choice(self.majority_idx, new_majority_size, replace=False)
      rand_majority_data = self.X[rand_majority_idx]
      rand_majority_label = [self.majority_label for i in rand_majority_data]
      rand_majority_weights = D_t[rand_majority_idx]
      X_new.extend(rand_majority_data)
      y_new.extend(rand_majority_label)
      D_new.extend(rand_majority_weights)
      # Create artificial examples from minority class using SMOTE
      # and add these examples to the new dataset
      create_num = new_minority_size - self.minority_size
      create_amount = (create_num/self.minority_size)*100
      synthetic_data = SMOTE(minority_data, create_amount, self.k)
      synthetic_label = np.full(synthetic_data.shape[0], dtype=np.int64,
                                fill_value=self.minority_label)
      synthetic_weights = np.full(synthetic_data.shape[0], dtype=np.float64,
                                  fill_value=(1./self.m))
      X_new.extend(synthetic_data)
      y_new.extend(synthetic_label)
      D_new.extend(synthetic_weights)
    else:
      # Add all majority samples to the new dataset
      majority_data = self.X[self.majority_idx]
      label_majority = [self.majority_label for i in majority_data]
      majority_weights = D_t[self.majority_idx]
      X_new.extend(majority_data)
      y_new.extend(label_majority)
      D_new.extend(majority_weights)
      # Take a random sample of size new_minority_size from minority class
      # and add the sample to the new dataset
      rand_minority_idx = np.random.choice(self.minority_idx, new_minority_size, replace=False)
      rand_minority_data = self.X[rand_minority_idx]
      rand_minority_label = [self.minority_label for i in rand_minority_data]
      rand_minority_weights = D_t[rand_minority_idx]
      X_new.extend(rand_minority_data)
      y_new.extend(rand_minority_label)
      D_new.extend(rand_minority_weights)
      # Create artificial examples from majority class using SMOTE
      # and add these examples to the new dataset
      create_num = new_majority_size - self.majority_size
      create_amount = (create_num/self.majority_size)*100
      synthetic_data = SMOTE(majority_data, create_amount, self.k)
      synthetic_label = np.full(synthetic_data.shape[0], dtype=np.int64,
                                fill_value=self.majority_label)
      synthetic_weights = np.full(synthetic_data.shape[0], dtype=np.float64,
                                  fill_value=(1./self.m))
      X_new.extend(synthetic_data)
      y_new.extend(synthetic_label)
      D_new.extend(synthetic_weights)
    
    return X_new, y_new, D_new

  

  # Running the RBBoost algorithm
  def run(self):

    # Initilize distribution
    D_t = (1./self.m) * np.ones(self.m)

    for t in trange(self.T):

      # Choose the new (temporary) training set
      X_new, y_new, D_new = self.RandomBalance(D_t)

      # Train a new classifier on training data with distibution D_t
      weak_learn = DecisionTreeClassifier(max_depth=max_depth, max_features=max_features)
      weak_learn.fit(X_new, y_new, sample_weight=D_new)
      
      # Get predictions and probabilities for the training data
      predictions = weak_learn.predict(self.X)
      probabilities = weak_learn.predict_proba(self.X)
      
      if t == 0:
        for index, value in enumerate(weak_learn.classes_):
          self.index_to_label[index] = value
          self.label_to_index[value] = index
      
      # Compute pseudo-loss
      pseudo_loss = 0

      # For each sample
      for index, true_label in enumerate(self.y):
        
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]

        # For each wrong label
        current_loss = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            current_loss += D_t[index] * (1 - true_prob + wrong_prob)
        # Add it to pseudo_loss
        pseudo_loss += current_loss

      # Final step to compute pseudo-loss
      pseudo_loss = (1/2)*pseudo_loss

      # Compute beta
      beta = pseudo_loss / (1 - pseudo_loss)
      
      # Update distribution
      for index, value in enumerate(D_t):
        predicted_label = predictions[index]
        true_label = self.y[index]
        # Probability of assigning the true label
        true_index = self.label_to_index[true_label]
        true_prob = probabilities[index][true_index]
        # Probability of assigning the wrong label
        power = 0
        for label, label_idx in self.label_to_index.items():
          if label != true_label:
            # Probability of assigning the wrong label
            wrong_prob = probabilities[index][label_idx]
            # Compute loss for the given sample
            power += (1 + true_prob - wrong_prob)
        power = (1/2) * power
        D_t[index] = (value) * pow(beta,power)
      
      # Normalization factor
      Z_t = np.sum(D_t)
      D_t = D_t / Z_t
      
      # Append weak learner to the ensemble
      self.H.append(weak_learn)
      # Append computed loss of the weak learner
      self.loss_list.append(pseudo_loss)
      # Append computed beta of the weak learner
      self.beta_list.append(beta)
  
      # print('t =', t, '--> pseudo-loss =', pseudo_loss)
    
    # Convert beta and loss lists to numpy array
    self.beta_list = np.array(self.beta_list)
    self.loss_list = np.array(self.loss_list)


  def predict(self, input):
    # Sum of the probabilities made by each hypothesis for a given sample and label
    sum_of_h = None
    # Compute the multiply factors
    alpha_list = np.log(1/self.beta_list)
    # For each hypothesis
    for index, h in enumerate(self.H):
      # Predict probabilities
      probabilities = h.predict_proba(input)
      # Multiply probabilities by the hypothesis computed alpha
      weighted_prob = alpha_list[index] * probabilities
      # If it's the first hypothesis
      if index == 0:
        sum_of_h = weighted_prob
      else:
        sum_of_h += weighted_prob
    # Choose the label using argmax
    predicted_idx = np.argmax(sum_of_h, axis=1)
    predicted_labels = []
    for pr in predicted_idx:
      predicted_labels.append(self.index_to_label[pr])
    predicted_labels = np.array(predicted_labels)
    #print(predicted_labels)
    # Remember the h_finite value
    h_fin = [sum_of_h[index][label]*predicted_labels[index] for index, label in enumerate(predicted_idx)]
    #print(h_fin)
    alpha_norm = np.linalg.norm(alpha_list, ord=1)
    h_fin = h_fin/alpha_norm
    return predicted_labels, h_fin

  # Print the computed pseudo-loss of each classifier
  def pseudo_losses(self):
    return self.loss_list


### 5.2) Run on 5-fold

In [36]:
# Remove last results
five_fold_results = five_fold_results[five_fold_results['Algorithm'] != 'RBBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  # Fold counter
  fold_count = 1
  # A list containing all the fold results
  all_fold_results = []
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # For each fold
  for train_index, test_index in five_fold.split(X):
    # Split the train and test set
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Create an RBBoost object
    rbboost = RBBoost(X_train, y_train, t, k_in_knn)
    # Run the algorithm
    rbboost.run()
    # Add scatter of this pseudo-loss to the loss plotter
    pseudo_losses = rbboost.pseudo_losses()
    loss_plotter.add_scatter(pseudo_losses, 'fold-'+str(fold_count))
    # Prediction on the test data
    predictions, hfin_values = rbboost.predict(X_test)
    model_margin = abs(hfin_values[0])
    for value in hfin_values:
      current_margin = abs(value)
      if current_margin < model_margin:
        model_margin = current_margin
    # Plot ROC curve
    auc_score = roc_plotter.add_scatter(y_test, hfin_values, 'Fold-'+str(fold_count))
    # Compute the loss on the test data
    loss = ensemble_loss_binary(y_test, predictions, hfin_values)
    results = {'Loss': loss, 'Margin': model_margin}
    results['AUC'] = auc_score
    results.update(get_metrics(y_test, predictions))
    all_fold_results.append(results)
    print('Fold-'+str(fold_count), results, '\n')
    fold_count += 1
  
  # Compute the average
  df_results = pd.DataFrame(all_fold_results)
  avg_results = dict(df_results.mean())
  # Print a horizontal line
  print ('-' * 128)

  # Remember the results
  avg_results['Algorithm'] = 'RBBoost'
  avg_results['T'] = t
  five_fold_results = five_fold_results.append(avg_results, ignore_index=True)
  print('Average', avg_results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()

T= 10
--------------------------------------------------------------------------------------------------------------------------------



Fold-1 {'Loss': 0.9345953580913248, 'Margin': 0.11345893972476302, 'AUC': 0.6380933477003345, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7483660130718954, 'TN': 229, 'FP': 0, 'FN': 77, 'TP': 0} 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.




Fold-2 {'Loss': 0.8506873902971941, 'Margin': 0.31097073368440764, 'AUC': 0.6064730436827561, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.761437908496732, 'TN': 233, 'FP': 0, 'FN': 73, 'TP': 0} 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.




Fold-4 {'Loss': 0.8956506669192434, 'Margin': 0.4321097462531895, 'AUC': 0.6948687412623622, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7091503267973857, 'TN': 217, 'FP': 0, 'FN': 89, 'TP': 0} 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.




Fold-5 {'Loss': 0.9087553858497646, 'Margin': 0.31999812505716063, 'AUC': 0.7212213970293055, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.6928104575163399, 'TN': 212, 'FP': 0, 'FN': 94, 'TP': 0} 

--------------------------------------------------------------------------------------------------------------------------------
Average {'Loss': 0.8717387626911621, 'Margin': 0.3811092992923413, 'AUC': 0.7190552293338071, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 225.0, 'FP': 0.0, 'FN': 81.0, 'TP': 0.0, 'Algorithm': 'RBBoost', 'T': 100}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



### 5.3) Run on whole training data

In [37]:
# Remove last results
train_results = train_results[train_results['Algorithm'] != 'RBBoost']

for t in T:

  # Print the pseudo-loss of each classifier
  loss_plotter = LossPlotter('Loss of each classifier for T='+str(t), 't', 'pseudo-loss')
  roc_plotter = RocPlotter('ROC curve for T='+str(t))
  print('T=',str(t))
  # Print a horizontal line
  print ('-' * 128)

  # Create an RBBoost object
  rbboost = RBBoost(X, y, t, k_in_knn)
  # Run the algorithm
  rbboost.run()
  # Add scatter of this pseudo-loss to the loss plotter
  pseudo_losses = rbboost.pseudo_losses()
  loss_plotter.add_scatter(pseudo_losses, 'train data')
  # Prediction on the test data
  predictions, hfin_values = rbboost.predict(X)
  model_margin = abs(hfin_values[0])
  for value in hfin_values:
    current_margin = abs(value)
    if current_margin < model_margin:
      model_margin = current_margin
  # Plot ROC curve
  auc_score = roc_plotter.add_scatter(y, hfin_values, 'train data')
  # Compute the loss on the test data
  loss = ensemble_loss_binary(y, predictions, hfin_values)
  # Compute metrics
  results = {'Loss': loss, 'Margin': model_margin}
  results.update(get_metrics(y, predictions))

  # Remember the results
  results['Algorithm'] = 'RBBoost'
  results['T'] = t
  results['AUC'] = auc_score
  train_results = train_results.append(results, ignore_index=True)
  print(results)

  # Show the loss plot
  loss_plotter.show()
  roc_plotter.show()
  plot_histogram(y, hfin_values, 'The histogram of scores compared to true labels for T='+str(t))

T= 10
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.9123619781528342, 'Margin': 0.1977001287178402, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 1125, 'FP': 0, 'FN': 405, 'TP': 0, 'Algorithm': 'RBBoost', 'T': 10, 'AUC': 0.6415637860082304}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



T= 50
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8884390894475666, 'Margin': 0.2263486398368169, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 1125, 'FP': 0, 'FN': 405, 'TP': 0, 'Algorithm': 'RBBoost', 'T': 50, 'AUC': 0.7338820301783264}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



T= 100
--------------------------------------------------------------------------------------------------------------------------------



{'Loss': 0.8803884429918041, 'Margin': 0.2899143447935746, 'Precision': 0.0, 'Recall': 0.0, 'F1score': 0.0, 'Accuracy': 0.7352941176470589, 'TN': 1125, 'FP': 0, 'FN': 405, 'TP': 0, 'Algorithm': 'RBBoost', 'T': 100, 'AUC': 0.7449382716049383}


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



## 6) SVM

### 6.1) Run on 5-fold

In [38]:
# Remove last results
five_fold_results = five_fold_results[five_fold_results['Algorithm'] != 'SVM']

roc_plotter = RocPlotter('ROC curve for T='+str(t))
# Fold counter
fold_count = 1
# A list containing all the fold results
all_fold_results = []
print('SVM')
# Print a horizontal line
print ('-' * 128)
# For each fold

for train_index, test_index in five_fold.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  svm_classifier = svm.SVC(probability=True)
  svm_classifier.fit(X_train, y_train)
  # Predict labels
  svm_pred_labels = svm_classifier.predict(X_test)
  # Predict probability
  svm_pred_prob = svm_classifier.predict_proba(X_test)
  svm_hfin_idx = np.argmax(svm_pred_prob, axis=1)
  svm_hfin = [pred[svm_hfin_idx[index]] for index, pred in enumerate(svm_pred_prob)]
  svm_hfin_label = [value * svm_pred_labels[index] for index, value in enumerate(svm_hfin)]
  # Compute the loss on the test data
  loss = ensemble_loss_binary(y_test, svm_pred_labels, svm_hfin_label)
  # Compute AUC score and ROC curve
  auc_score = roc_plotter.add_scatter(y_test, svm_hfin_label, 'train data')
  # Model margin
  model_margin = abs(svm_hfin_label[0])
  for value in svm_hfin_label:
    current_margin = abs(value)
    if current_margin < model_margin:
      model_margin = current_margin
  results = {'Loss': loss, 'Margin': model_margin}
  results['AUC'] = auc_score
  results.update(get_metrics(y_test, svm_pred_labels))
  all_fold_results.append(results)
  print('Fold-'+str(fold_count), results, '\n')
  fold_count += 1

# Compute the average
df_results = pd.DataFrame(all_fold_results)
avg_results = dict(df_results.mean())
# Print a horizontal line
print ('-' * 128)

# Remember the results
avg_results['Algorithm'] = 'SVM'
five_fold_results = five_fold_results.append(avg_results, ignore_index=True)
print('Average', avg_results)

# Show the loss plot
roc_plotter.show()

SVM
--------------------------------------------------------------------------------------------------------------------------------
Fold-1 {'Loss': 0.8138422594388125, 'Margin': 0.5, 'AUC': 0.7742868485226564, 'Precision': 0.6774193548387096, 'Recall': 0.2727272727272727, 'F1score': 0.38888888888888884, 'Accuracy': 0.7843137254901961, 'TN': 219, 'FP': 10, 'FN': 56, 'TP': 21} 

Fold-2 {'Loss': 0.8353415247823766, 'Margin': 0.5053071918003392, 'AUC': 0.7832618025751072, 'Precision': 0.5151515151515151, 'Recall': 0.2328767123287671, 'F1score': 0.320754716981132, 'Accuracy': 0.7647058823529411, 'TN': 217, 'FP': 16, 'FN': 56, 'TP': 17} 

Fold-3 {'Loss': 0.7396988969163049, 'Margin': 0.5058800003771968, 'AUC': 0.8162689933523267, 'Precision': 0.8064516129032258, 'Recall': 0.3472222222222222, 'F1score': 0.48543689320388356, 'Accuracy': 0.826797385620915, 'TN': 228, 'FP': 6, 'FN': 47, 'TP': 25} 

Fold-4 {'Loss': 0.8567484614901683, 'Margin': 0.5, 'AUC': 0.7742712162791903, 'Precision': 0.7666

### 6.2) Run on whole training data

In [39]:
# Remove last results
train_results = train_results[train_results['Algorithm'] != 'SVM']

roc_plotter = RocPlotter('ROC curve for T='+str(t))
# Fold counter
fold_count = 1
# A list containing all the fold results
all_fold_results = []
print('SVM')
# Print a horizontal line
print ('-' * 128)
# For each fold

# Train a SVM classifier on data
svm_classifier = svm.SVC(probability=True)
svm_classifier.fit(X, y)

# Predict labels
svm_pred_labels = svm_classifier.predict(X)
# Predict probability
svm_pred_prob = svm_classifier.predict_proba(X)
svm_hfin_idx = np.argmax(svm_pred_prob, axis=1)
svm_hfin = [pred[svm_hfin_idx[index]] for index, pred in enumerate(svm_pred_prob)]
svm_hfin_label = [value * svm_pred_labels[index] for index, value in enumerate(svm_hfin)]
# Compute the loss on the test data
loss = ensemble_loss_binary(y, svm_pred_labels, svm_hfin_label)
# Compute AUC score and ROC curve
auc_score = roc_plotter.add_scatter(y, svm_hfin_label, 'train data')
# Model margin
model_margin = abs(svm_hfin_label[0])
for value in svm_hfin_label:
  current_margin = abs(value)
  if current_margin < model_margin:
    model_margin = current_margin
# Remember the results
results = {'Loss': loss, 'Margin': model_margin}
results.update(get_metrics(y, svm_pred_labels))
all_fold_results.append(results)
# Remember the results
results['Algorithm'] = 'SVM'
results['AUC'] = auc_score
train_results = train_results.append(results, ignore_index=True)
print(results)
# Show the loss plot
roc_plotter.show()

SVM
--------------------------------------------------------------------------------------------------------------------------------
{'Loss': 0.8207398837493515, 'Margin': 0.5, 'Precision': 0.696969696969697, 'Recall': 0.2839506172839506, 'F1score': 0.4035087719298246, 'Accuracy': 0.7777777777777778, 'TN': 1075, 'FP': 50, 'FN': 290, 'TP': 115, 'Algorithm': 'SVM', 'AUC': 0.7917146776406035}


In [40]:
plot_histogram(y, svm_hfin_label, 'The histogram of scores compared to true label')

## 7) Results

The results of implemented models shown in this section are based on different metrics. In the 5-fold method, the algorithm trained on 4-fold of the data and tested on the other remaining fold. In the other method, both training and evaluating steps rely on the training data.

### 7.1) 5-Fold results

In [41]:
five_fold_results

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
0,AdaBoostM2,10,0.874355,0.000000,0.000000,0.000000,0.703633,0.735294,225.0,0.0,81.0,0.0,0.557860
1,AdaBoostM2,50,0.868800,0.000000,0.000000,0.000000,0.738481,0.735294,225.0,0.0,81.0,0.0,0.510457
2,AdaBoostM2,100,0.854806,0.817229,0.098126,0.162762,0.744424,0.747712,221.0,4.0,73.2,7.8,0.502756
3,RUSBoost,10,0.925550,0.450684,0.562218,0.497281,0.705282,0.704575,169.8,55.2,35.2,45.8,0.523897
4,RUSBoost,50,0.909704,0.462913,0.616407,0.528234,0.740953,0.711111,167.6,57.4,31.0,50.0,0.505653
5,RUSBoost,100,0.902923,0.467260,0.585562,0.517120,0.744772,0.716340,171.4,53.6,33.2,47.8,0.501905
6,SMOTEBoost,10,0.872238,0.513412,0.481033,0.495129,0.690505,0.743791,188.6,36.4,42.0,39.0,0.514869
7,SMOTEBoost,50,0.872481,0.511680,0.510677,0.505282,0.716483,0.740523,185.4,39.6,39.8,41.2,0.501762
8,SMOTEBoost,100,0.847256,0.560584,0.472263,0.512051,0.720444,0.764052,195.4,29.6,42.6,38.4,0.504967
9,RBBoost,10,0.915870,0.000000,0.000000,0.000000,0.657557,0.735294,225.0,0.0,81.0,0.0,0.254111


#### 7.1.1) Sort by Recall

Sort the results based on the number of Recall metric.

In [42]:
five_fold_results.sort_values(by=['Recall'], ascending=False)

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
4,RUSBoost,50,0.909704,0.462913,0.616407,0.528234,0.740953,0.711111,167.6,57.4,31.0,50.0,0.505653
5,RUSBoost,100,0.902923,0.467260,0.585562,0.517120,0.744772,0.716340,171.4,53.6,33.2,47.8,0.501905
3,RUSBoost,10,0.925550,0.450684,0.562218,0.497281,0.705282,0.704575,169.8,55.2,35.2,45.8,0.523897
7,SMOTEBoost,50,0.872481,0.511680,0.510677,0.505282,0.716483,0.740523,185.4,39.6,39.8,41.2,0.501762
6,SMOTEBoost,10,0.872238,0.513412,0.481033,0.495129,0.690505,0.743791,188.6,36.4,42.0,39.0,0.514869
8,SMOTEBoost,100,0.847256,0.560584,0.472263,0.512051,0.720444,0.764052,195.4,29.6,42.6,38.4,0.504967
12,SVM,NaN,0.831158,0.678852,0.269059,0.384544,0.781816,0.771895,214.6,10.4,59.4,21.6,0.506419
2,AdaBoostM2,100,0.854806,0.817229,0.098126,0.162762,0.744424,0.747712,221.0,4.0,73.2,7.8,0.502756
0,AdaBoostM2,10,0.874355,0.000000,0.000000,0.000000,0.703633,0.735294,225.0,0.0,81.0,0.0,0.557860
1,AdaBoostM2,50,0.868800,0.000000,0.000000,0.000000,0.738481,0.735294,225.0,0.0,81.0,0.0,0.510457


#### 7.1.2) Sort by F-score

Sort the results based on the F-score metric.

In [43]:
five_fold_results.sort_values(by=['F1score'], ascending=False)

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
4,RUSBoost,50,0.909704,0.462913,0.616407,0.528234,0.740953,0.711111,167.6,57.4,31.0,50.0,0.505653
5,RUSBoost,100,0.902923,0.467260,0.585562,0.517120,0.744772,0.716340,171.4,53.6,33.2,47.8,0.501905
8,SMOTEBoost,100,0.847256,0.560584,0.472263,0.512051,0.720444,0.764052,195.4,29.6,42.6,38.4,0.504967
7,SMOTEBoost,50,0.872481,0.511680,0.510677,0.505282,0.716483,0.740523,185.4,39.6,39.8,41.2,0.501762
3,RUSBoost,10,0.925550,0.450684,0.562218,0.497281,0.705282,0.704575,169.8,55.2,35.2,45.8,0.523897
6,SMOTEBoost,10,0.872238,0.513412,0.481033,0.495129,0.690505,0.743791,188.6,36.4,42.0,39.0,0.514869
12,SVM,NaN,0.831158,0.678852,0.269059,0.384544,0.781816,0.771895,214.6,10.4,59.4,21.6,0.506419
2,AdaBoostM2,100,0.854806,0.817229,0.098126,0.162762,0.744424,0.747712,221.0,4.0,73.2,7.8,0.502756
0,AdaBoostM2,10,0.874355,0.000000,0.000000,0.000000,0.703633,0.735294,225.0,0.0,81.0,0.0,0.557860
1,AdaBoostM2,50,0.868800,0.000000,0.000000,0.000000,0.738481,0.735294,225.0,0.0,81.0,0.0,0.510457


#### 7.1.3) Sort by Loss

Sort the results based on the value of Loss.

In [44]:
five_fold_results.sort_values(by=['Loss'])

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
12,SVM,NaN,0.831158,0.678852,0.269059,0.384544,0.781816,0.771895,214.6,10.4,59.4,21.6,0.506419
8,SMOTEBoost,100,0.847256,0.560584,0.472263,0.512051,0.720444,0.764052,195.4,29.6,42.6,38.4,0.504967
2,AdaBoostM2,100,0.854806,0.817229,0.098126,0.162762,0.744424,0.747712,221.0,4.0,73.2,7.8,0.502756
1,AdaBoostM2,50,0.868800,0.000000,0.000000,0.000000,0.738481,0.735294,225.0,0.0,81.0,0.0,0.510457
11,RBBoost,100,0.871739,0.000000,0.000000,0.000000,0.719055,0.735294,225.0,0.0,81.0,0.0,0.381109
6,SMOTEBoost,10,0.872238,0.513412,0.481033,0.495129,0.690505,0.743791,188.6,36.4,42.0,39.0,0.514869
7,SMOTEBoost,50,0.872481,0.511680,0.510677,0.505282,0.716483,0.740523,185.4,39.6,39.8,41.2,0.501762
0,AdaBoostM2,10,0.874355,0.000000,0.000000,0.000000,0.703633,0.735294,225.0,0.0,81.0,0.0,0.557860
10,RBBoost,50,0.880056,0.000000,0.000000,0.000000,0.714337,0.735294,225.0,0.0,81.0,0.0,0.387180
5,RUSBoost,100,0.902923,0.467260,0.585562,0.517120,0.744772,0.716340,171.4,53.6,33.2,47.8,0.501905


#### 7.1.4) Precision/Recall Plot

In [45]:
fig = go.Figure(data=[
    go.Bar(name='Precision', x=five_fold_results['Algorithm'], y=five_fold_results['Precision']),
    go.Bar(name='Recall', x=five_fold_results['Algorithm'], y=five_fold_results['Recall'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

### 7.2) Training results

In [46]:
train_results

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
0,AdaBoostM2,10,0.867805,0.000000,0.000000,0.000000,0.732675,0.735294,1125,0,405,0,0.531634
1,AdaBoostM2,50,0.866344,0.000000,0.000000,0.000000,0.751824,0.735294,1125,0,405,0,0.503479
2,AdaBoostM2,100,0.846069,0.875000,0.086420,0.157303,0.759342,0.754902,1120,5,370,35,0.505792
3,RUSBoost,10,0.933031,0.446281,0.666667,0.534653,0.726557,0.692810,790,335,135,270,0.506989
4,RUSBoost,50,0.903500,0.473214,0.654321,0.549223,0.747435,0.715686,830,295,140,265,0.503675
5,RUSBoost,100,0.891899,0.485437,0.617284,0.543478,0.760658,0.725490,860,265,155,250,0.503698
6,SMOTEBoost,10,0.871641,0.513158,0.481481,0.496815,0.708176,0.741830,940,185,210,195,0.516459
7,SMOTEBoost,50,0.845750,0.569231,0.456790,0.506849,0.721180,0.764706,985,140,220,185,0.506898
8,SMOTEBoost,100,0.838741,0.582090,0.481481,0.527027,0.731276,0.771242,985,140,210,195,0.508578
9,RBBoost,10,0.912362,0.000000,0.000000,0.000000,0.641564,0.735294,1125,0,405,0,0.197700


#### 7.2.1) Sort by Recall

Sort the results based on the number of Recall metric.

In [47]:
train_results.sort_values(by=['Recall'], ascending=False)

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
3,RUSBoost,10,0.933031,0.446281,0.666667,0.534653,0.726557,0.692810,790,335,135,270,0.506989
4,RUSBoost,50,0.903500,0.473214,0.654321,0.549223,0.747435,0.715686,830,295,140,265,0.503675
5,RUSBoost,100,0.891899,0.485437,0.617284,0.543478,0.760658,0.725490,860,265,155,250,0.503698
6,SMOTEBoost,10,0.871641,0.513158,0.481481,0.496815,0.708176,0.741830,940,185,210,195,0.516459
8,SMOTEBoost,100,0.838741,0.582090,0.481481,0.527027,0.731276,0.771242,985,140,210,195,0.508578
7,SMOTEBoost,50,0.845750,0.569231,0.456790,0.506849,0.721180,0.764706,985,140,220,185,0.506898
12,SVM,NaN,0.820740,0.696970,0.283951,0.403509,0.791715,0.777778,1075,50,290,115,0.500000
2,AdaBoostM2,100,0.846069,0.875000,0.086420,0.157303,0.759342,0.754902,1120,5,370,35,0.505792
0,AdaBoostM2,10,0.867805,0.000000,0.000000,0.000000,0.732675,0.735294,1125,0,405,0,0.531634
1,AdaBoostM2,50,0.866344,0.000000,0.000000,0.000000,0.751824,0.735294,1125,0,405,0,0.503479


#### 7.2.2) Sort by F-score

Sort the results based on the F-score metric.

In [48]:
train_results.sort_values(by=['F1score'], ascending=False)

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
4,RUSBoost,50,0.903500,0.473214,0.654321,0.549223,0.747435,0.715686,830,295,140,265,0.503675
5,RUSBoost,100,0.891899,0.485437,0.617284,0.543478,0.760658,0.725490,860,265,155,250,0.503698
3,RUSBoost,10,0.933031,0.446281,0.666667,0.534653,0.726557,0.692810,790,335,135,270,0.506989
8,SMOTEBoost,100,0.838741,0.582090,0.481481,0.527027,0.731276,0.771242,985,140,210,195,0.508578
7,SMOTEBoost,50,0.845750,0.569231,0.456790,0.506849,0.721180,0.764706,985,140,220,185,0.506898
6,SMOTEBoost,10,0.871641,0.513158,0.481481,0.496815,0.708176,0.741830,940,185,210,195,0.516459
12,SVM,NaN,0.820740,0.696970,0.283951,0.403509,0.791715,0.777778,1075,50,290,115,0.500000
2,AdaBoostM2,100,0.846069,0.875000,0.086420,0.157303,0.759342,0.754902,1120,5,370,35,0.505792
0,AdaBoostM2,10,0.867805,0.000000,0.000000,0.000000,0.732675,0.735294,1125,0,405,0,0.531634
1,AdaBoostM2,50,0.866344,0.000000,0.000000,0.000000,0.751824,0.735294,1125,0,405,0,0.503479


#### 7.2.3) Sort by Loss

Sort the results based on the value of Loss.

In [49]:
train_results.sort_values(by=['Loss'])

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
12,SVM,NaN,0.820740,0.696970,0.283951,0.403509,0.791715,0.777778,1075,50,290,115,0.500000
8,SMOTEBoost,100,0.838741,0.582090,0.481481,0.527027,0.731276,0.771242,985,140,210,195,0.508578
7,SMOTEBoost,50,0.845750,0.569231,0.456790,0.506849,0.721180,0.764706,985,140,220,185,0.506898
2,AdaBoostM2,100,0.846069,0.875000,0.086420,0.157303,0.759342,0.754902,1120,5,370,35,0.505792
1,AdaBoostM2,50,0.866344,0.000000,0.000000,0.000000,0.751824,0.735294,1125,0,405,0,0.503479
0,AdaBoostM2,10,0.867805,0.000000,0.000000,0.000000,0.732675,0.735294,1125,0,405,0,0.531634
6,SMOTEBoost,10,0.871641,0.513158,0.481481,0.496815,0.708176,0.741830,940,185,210,195,0.516459
11,RBBoost,100,0.880388,0.000000,0.000000,0.000000,0.744938,0.735294,1125,0,405,0,0.289914
10,RBBoost,50,0.888439,0.000000,0.000000,0.000000,0.733882,0.735294,1125,0,405,0,0.226349
5,RUSBoost,100,0.891899,0.485437,0.617284,0.543478,0.760658,0.725490,860,265,155,250,0.503698


In [50]:
train_results.sort_values(by=['Loss'])

,Algorithm,T,Loss,Precision,Recall,F1score,AUC,Accuracy,TN,FP,FN,TP,Margin
12,SVM,NaN,0.820740,0.696970,0.283951,0.403509,0.791715,0.777778,1075,50,290,115,0.500000
8,SMOTEBoost,100,0.838741,0.582090,0.481481,0.527027,0.731276,0.771242,985,140,210,195,0.508578
7,SMOTEBoost,50,0.845750,0.569231,0.456790,0.506849,0.721180,0.764706,985,140,220,185,0.506898
2,AdaBoostM2,100,0.846069,0.875000,0.086420,0.157303,0.759342,0.754902,1120,5,370,35,0.505792
1,AdaBoostM2,50,0.866344,0.000000,0.000000,0.000000,0.751824,0.735294,1125,0,405,0,0.503479
0,AdaBoostM2,10,0.867805,0.000000,0.000000,0.000000,0.732675,0.735294,1125,0,405,0,0.531634
6,SMOTEBoost,10,0.871641,0.513158,0.481481,0.496815,0.708176,0.741830,940,185,210,195,0.516459
11,RBBoost,100,0.880388,0.000000,0.000000,0.000000,0.744938,0.735294,1125,0,405,0,0.289914
10,RBBoost,50,0.888439,0.000000,0.000000,0.000000,0.733882,0.735294,1125,0,405,0,0.226349
5,RUSBoost,100,0.891899,0.485437,0.617284,0.543478,0.760658,0.725490,860,265,155,250,0.503698


#### 7.2.4) Precision/Recall Plot

In [51]:
fig = go.Figure(data=[
    go.Bar(name='Precision', x=train_results['Algorithm'], y=train_results['Precision']),
    go.Bar(name='Recall', x=train_results['Algorithm'], y=train_results['Recall'])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

## 8) Conclusion

First of all, we should consider that the dataset used in this project, Haberman, is a hard dataset [[10]](https://arxiv.org/ftp/arxiv/papers/1703/1703.08283.pdf). As the referenced paper proved, many classification methods couldn't achieve well-done results in terms of AUC and F-score metrics. Also, we can expect this complexity for the Haberman dataset if we look at the data shown in section 1.2, it has just three features and the positive and negative examples are really mixed together.

As the data relates to a disease detection task, and the minority class labeled as positve, the Recall metric is crucial for us. The tables in section 7.1.1 and 7.2.1 show that the RUSBoost with any chosen T outperforms any other algorithms in terms of Recall. RUSBoost also achieves a better F1-score than other algorithms. After RUSBOOST, the SMOTEBoost achieves noticeable results based on most metrics.

SVM performs better in terms of precision, which means that SVM reaches a better classifier on the majority class. So it's perfectly shown that SVM couldn't consider the imbalance between the data classes. But in general, it outperforms the AdaBoost.M2 for this specific dataset. It also predicts the training samples with higher scores, as shown in the histogram plot of the 6.2 section.

RBBoost performs better when the number of classifiers in the ensemble (T) increases. It also has a lower loss than most of the algorithms. But as sometimes happens that Random Balance makes artificial samples from the majority class, this method doesn't reach a better performance than the RUSBosst and SMOTEBoost.

The plot in the 7.1.4 and 7.2.4 sections shows that in RUSBoost, we had a greater recall than precision. The reason is RUSBoost doesn't consider many of the majority data in each iteration, so it focuses on minority data. But if we want a trade-off between precision and recall, SMOTEBoost performs reasonably well. It considers all the majority data, and also over sampled minority data.